In [3]:
import geopandas as gpd
import pandas as pd
from time import sleep
from shapely.geometry import mapping
from geopy.distance import distance
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
city_name = 'Санкт-Петербург'
common_crs = 3857

In [5]:
blocks = gpd.read_file(f'/home/gk/jupyter/accessibility_paper/districts_splitted_{city_name}.geojson').to_crs(common_crs)

DriverError: districts_splitted_Санкт-Петербург.geojson: No such file or directory

In [5]:
blocks['centroids'] = blocks['geometry'].centroid
blocks.drop(columns=['geometry'], inplace=True)
blocks.rename(columns={'centroids':'geometry'}, inplace=True)


if 'id' not in blocks.columns:
    blocks.reset_index(inplace=True, drop=False)
    blocks.rename(columns={'index': 'block_id'}, inplace=True)
else:
    blocks.rename(columns={'id': 'block_id'}, inplace=True)

/tmp/ipykernel_726430/3777901727.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  blocks['centroids'] = blocks['geometry'].centroid


In [6]:
def get_coords(row):

    val = tuple([row['geometry'].x, row['geometry'].y])
    return val

blocks['coords'] = blocks.apply(lambda row: get_coords(row), axis=1)
blocks

,block_id,geometry,coords
0,2,POINT (30.27056 60.00945),"(30.27055879167984, 60.009453184430605)"
1,3,POINT (30.36738 59.92924),"(30.367375299152457, 59.92923650637079)"
2,4,POINT (30.33456 59.92050),"(30.334557218590366, 59.92050001748285)"
3,5,POINT (30.30166 59.96874),"(30.30165795411099, 59.968735899716926)"
4,6,POINT (30.36209 59.94193),"(30.36209035082577, 59.94193078749883)"
...,...,...,...
5572,6306,POINT (30.29904 59.93184),"(30.299036249539785, 59.931840726257356)"
5573,6307,POINT (30.35935 59.94052),"(30.359350690583135, 59.940524891633565)"
5574,6308,POINT (30.30147 59.91323),"(30.301471851446788, 59.91323389039739)"
5575,6309,POINT (30.31809 60.01764),"(30.31808970321377, 60.017644624547394)"


In [7]:
blocks.coords.str[0]

0       30.270559
1       30.367375
2       30.334557
3       30.301658
4       30.362090
          ...    
5572    30.299036
5573    30.359351
5574    30.301472
5575    30.318090
5576    30.303901
Name: coords, Length: 5577, dtype: float64

In [8]:
# df = pd.DataFrame({'x': blocks.coords.str[0].astype(float), 
#                   'y':blocks.coords.str[1].astype(float)
#                   }, index = blocks['block_id'])

In [9]:
# df.dropna(inplace=True)
# df

In [10]:
df = pd.DataFrame(columns=blocks['block_id'], index=blocks['block_id'])

In [11]:
df.head()

block_id,2,3,4,5,6,7,8,10,11,12,...,6301,6302,6303,6304,6305,6306,6307,6308,6309,6310
block_id,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
import numpy as np

def get_coordinates(blocks, current_block) -> tuple:
    coords = blocks.loc[blocks['block_id'] == current_block, 'coords'].squeeze()
    return coords


def get_dist(source, target):
    dist = round(distance(source, target).m, 0)
    return dist


def calc_dist(current_block) -> None:
    print('Осталось блоков: ', len(block_ids), end="\r")
    del block_ids[0]
    
    source = get_coordinates(blocks, current_block)
    targets = np.array(blocks.loc[blocks['block_id'].isin(block_ids)].coords)
    lst = [get_dist(source, t) for t in targets]
    df.loc[block_ids, current_block] = lst
    
    
block_ids = list(blocks['block_id'])

func = np.vectorize(lambda x: calc_dist(x))
func(block_ids)

print(df.head())
    
df.to_pickle(f'euclidean_{city_name}.pkl')

block_id   2     3     4     5    6    7    8    10   11   12    ... 6301  \
block_id                                                         ...        
2           NaN   NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
3         13218   NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
4         11115  3734   NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
5          5218  8216  5901   NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   
6         12047  1354  3683  7178  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN   

block_id 6302 6303 6304 6305 6306 6307 6308 6309 6310  
block_id                                               
2         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
5         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 5577 columns]


In [13]:
# --------------------------------------------------------------------------------

In [14]:
df

block_id,2,3,4,5,6,7,8,10,11,12,...,6301,6302,6303,6304,6305,6306,6307,6308,6309,6310
block_id,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11115,3734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5218,8216,5901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,12047,1354,3683,7178,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6306,8107,7580,4086,3561,7057,11273,93988,5954,12315,3585,...,21969,12994,10117,10370,15827,NaN,NaN,NaN,NaN,NaN
6307,11868,1403,3356,6947,332,11361,100199,12387,5594,9065,...,27965,9002,16210,16581,11997,6738,NaN,NaN,NaN,NaN
6308,9871,7466,3734,5339,7265,13006,94769,6958,12489,2241,...,21138,14203,11229,11400,17115,1810,6932,NaN,NaN,NaN


In [15]:
1/0

ZeroDivisionError: division by zero

In [ ]:
import rpyc
rpyc_connect = rpyc.connect('10.32.1.65', 18867, 
                           config={
                               'allow_public_attrs':True,
                               'allow_pickle':True
                           })
rpyc_connect._config['sync_request_timeout']=None

In [ ]:
import pickle

In [ ]:
G = pickle.loads(rpyc_connect.root.get_city_model_attr(
    "Saint_Petersburg", "MobilityGraph"))

In [ ]:
G.nodes

NodeView((0, 27365, 1, 1736, 2, 82612, 3, 81881, 4, 17, 38021, 38023, 5, 8437, 8433, 490, 6, 7, 32487, 104313, 182837, 38010, 8, 84242, 24138, 4135, 9, 4537, 10, 144709, 11, 9598, 12, 140718, 13, 117682, 14, 130222, 15, 130221, 16, 130218, 38024, 38020, 18, 722, 467, 10563, 62147, 19, 59289, 20, 119226, 50316, 97310, 119307, 21, 72547, 22, 81689, 23, 73186, 24, 45923, 2647, 45922, 25, 73189, 26, 5768, 88369, 27, 7576, 28, 97508, 29, 276747, 80097, 121711, 30, 82830, 31, 73547, 32, 154789, 87735, 5921, 73, 33, 25142, 154762, 52904, 34, 23588, 2143, 2144, 35, 142517, 36, 3214, 1994, 123263, 160361, 37, 59904, 38, 154031, 142823, 66, 910, 39, 185625, 7151, 83946, 40, 59899, 41, 115550, 329, 7153, 42, 59895, 43, 127828, 44, 128465, 45, 135410, 98799, 135408, 98798, 46, 118799, 98803, 127830, 47, 68925, 48, 66993, 49, 71369, 50, 16770, 51, 134487, 18977, 80521, 52, 18978, 72783, 99916, 53, 142792, 54, 88572, 55, 80498, 56, 134412, 134401, 134413, 57, 47492, 58, 8131, 59, 41043, 60, 6401, 61

In [ ]:
import networkx as nx
nx.get_node_attributes(G, 1)

{}

In [ ]:
from scipy import spatial
h_dist, h_nodes = spatial.cKDTree(G).query(df)

ValueError: data must be 2 dimensions

In [ ]:
import sys
import os
import numpy as np

# path to folder to CityGeoTools
folder = "/home/gk/jupyter/folder/CityGeoTools/CityGeoTools"
sys.path.append(folder)

from metrics.data import CityInformationModel as BaseModel
from metrics.calculations import CityMetricsMethods as CityMetrics

/home/gk/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/home/gk/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/gk/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/gk/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/gk/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarn

In [ ]:
from metrics.data import CityInformationModel as BaseModel
os.environ['RPYC_SERVER']="10.32.1.65:18867"
os.environ['POSTGRES']="5432"
city_model = BaseModel.CityInformationModel(
    city_name="Saint_Petersburg", city_crs=32636, cities_db_id=1, mode="general_mode")

Saint_Petersburg MobilityGraph
Saint_Petersburg Buildings
Saint_Petersburg Services
Saint_Petersburg PublicTransportStops
Saint_Petersburg ServiceTypes
Saint_Petersburg Blocks
Saint_Petersburg Municipalities
Saint_Petersburg AdministrativeUnits


In [ ]:
god_table = city_model.nk_attrs

In [ ]:
god_table.to_csv('god_table.csv')

In [ ]:
from scipy import spatial
h_dist, h_nodes = spatial.cKDTree(G).query(df)